In [92]:
import random
import numpy as np

# Configuração do Problema
# Ordens de Serviço (OS): (id, tipo, nível necessário, prazo)
# ordens_servico = [
#     (1, "mecanica", 3, 2),
#     (2, "eletrica", 2, 1),
#     (3, "mecanica", 2, 3),
#     (4, "eletrica", 1, 2),
#     (5, "mecanica", 1, 1),
# ]
# Lista de todas as habilidades possíveis
habilidades_possiveis = ["mecanica", "eletrica", "hidraulica"]

# Ordens de Serviço (OS): (id, tipo, nível necessário, prazo)
# Função para gerar ordens de serviço
def gerar_ordens_servico(quantidade=100):
    ordens = []
    id_os = 1
    
    # Gerar todas as ordens com uma única habilidade
    for _ in range(quantidade):
        habilidade = random.choice(habilidades_possiveis)
        nivel = random.randint(1, 5)
        prazo = random.randint(1, 5)
        ordens.append((id_os, habilidade, nivel, prazo))
        id_os += 1
    
    return ordens

# Substituir a definição atual das ordens de serviço por:
ordens_servico = gerar_ordens_servico(100)

# Substituir a definição atual das ordens de serviço por:
ordens_servico = gerar_ordens_servico(100)

In [93]:
# Operadores: (id, habilidades, nível máximo, disponibilidade em dias)
# operadores = [
#     (1, ["mecanica"], 3, 5),
#     (2, ["eletrica"], 2, 4),
#     (3, ["mecanica", "eletrica"], 2, 3),
# ]
# Função para gerar operadores
def gerar_operadores(quantidade=50):
    operadores = []
    id_operador = 1
    
    # Cálculo da quantidade para cada grupo
    qtd_tres_habilidades = int(quantidade * 0.15)  # 15% com 3 habilidades
    qtd_duas_habilidades = int(quantidade * 0.35)  # 35% com 2 habilidades
    qtd_uma_habilidade = quantidade - qtd_tres_habilidades - qtd_duas_habilidades  # restante com 1 habilidade
    
    # Operadores com 3 habilidades
    for _ in range(qtd_tres_habilidades):
        nivel = random.randint(1, 5)
        disponibilidade = random.randint(1, 5)
        operadores.append((id_operador, habilidades_possiveis.copy(), nivel, disponibilidade))
        id_operador += 1
    
    # Operadores com 2 habilidades
    for _ in range(qtd_duas_habilidades):
        habilidades = random.sample(habilidades_possiveis, 2)
        nivel = random.randint(1, 5)
        disponibilidade = random.randint(1, 5)
        operadores.append((id_operador, habilidades, nivel, disponibilidade))
        id_operador += 1
    
    # Operadores com 1 habilidade
    for _ in range(qtd_uma_habilidade):
        habilidade = [random.choice(habilidades_possiveis)]
        nivel = random.randint(1, 5)
        disponibilidade = random.randint(1, 5)
        operadores.append((id_operador, habilidade, nivel, disponibilidade))
        id_operador += 1
    
    return operadores

# Substituir a definição atual dos operadores por:
operadores = gerar_operadores(50)

In [94]:
# Representação do Cromossomo
def gerar_cromossomo():
    return [random.choice(range(len(operadores))) for _ in ordens_servico]

In [95]:
# Função de Fitness
def calcular_fitness(cromossomo):
    fitness = 0
    penalidades = {
        'habilidade': -100,     # Penalidade maior para incompatibilidade de habilidade
        'nivel': -30,           # Penalidade média para nível inadequado
        'disponibilidade': -20, # Penalidade menor para disponibilidade
        'match_perfeito': 150   # Recompensa maior para match perfeito
    }

    for os_index, operador_index in enumerate(cromossomo):
        os_tipo, os_nivel, os_prazo = ordens_servico[os_index][1:]
        op_habilidades, op_nivel_max, op_disponibilidade = operadores[operador_index][1:]

        # Penalidade se o operador não tem a habilidade necessária
        if os_tipo not in op_habilidades:
            fitness -= penalidades['habilidade']

        # Penalidade se o operador não atinge o nível necessário
        if os_nivel > op_nivel_max:
            fitness -= penalidades['nivel']

        # Penalidade se o operador não tem disponibilidade suficiente
        if os_prazo > op_disponibilidade:
            fitness -= penalidades['disponibilidade']

        # Recompensa se tudo está correto
        if (os_tipo in op_habilidades 
            and os_nivel <= op_nivel_max 
            and os_prazo <= op_disponibilidade):
            fitness += penalidades['match_perfeito']

    return fitness

In [96]:

# Operadores Genéticos
def selecao_torneio(populacao, fitnesses, tamanho=3):
    selecionados = random.sample(list(zip(populacao, fitnesses)), tamanho)
    return max(selecionados, key=lambda x: x[1])[0]

In [97]:
def crossover(pai1, pai2):
    ponto = random.randint(1, len(pai1) - 1)
    filho1 = pai1[:ponto] + pai2[ponto:]
    filho2 = pai2[:ponto] + pai1[ponto:]
    return filho1, filho2

In [98]:
def mutacao(cromossomo, taxa_mutacao=0.1):
    for i in range(len(cromossomo)):
        if random.random() < taxa_mutacao:
            cromossomo[i] = random.choice(range(len(operadores)))
    return cromossomo

In [99]:
# Algoritmo Genético
def algoritmo_genetico(geracoes=50, populacao_tamanho=20, taxa_mutacao=0.1, usar_elitismo=True):
    print("\n" + "="*50)
    print("INICIANDO OTIMIZAÇÃO".center(50))
    print("="*50)
    
    print(f"Iniciando algoritmo genético com {geracoes} gerações")
    print(f"Estratégia: {'Elitismo' if usar_elitismo else 'Sorteio'}")
    melhor_fitness_historico = []
    melhor_fitness_global = float('-inf')
    
    # Gerar população inicial
    populacao = [gerar_cromossomo() for _ in range(populacao_tamanho)]

    for geracao in range(geracoes):
        # Avaliar a população
        fitnesses = [calcular_fitness(cromo) for cromo in populacao]
        melhor_fitness_atual = max(fitnesses)
        melhor_fitness_historico.append(melhor_fitness_atual)

        if melhor_fitness_atual > melhor_fitness_global:
            melhor_fitness_global = melhor_fitness_atual
            print(f"[Geração {geracao:5d}] População: {len(populacao):3d} | "
                  f"Taxa Mutação: {taxa_mutacao:.2%} | "
                  f"Melhor Fitness: {melhor_fitness_atual:5d} | "
                  f"Melhoria: +{melhor_fitness_atual - melhor_fitness_historico[-2] if len(melhor_fitness_historico) > 1 else 0}")

        # Seleção e reprodução
        nova_populacao = []
        
        if usar_elitismo:
            # Mantém o melhor indivíduo
            elite_index = fitnesses.index(max(fitnesses))
            nova_populacao.append(populacao[elite_index])
        
        # Preenche o resto da população
        while len(nova_populacao) < populacao_tamanho:
            if usar_elitismo:
                pai1 = selecao_torneio(populacao, fitnesses)
                pai2 = selecao_torneio(populacao, fitnesses)
            else:
                # Seleção aleatória
                pai1 = random.choice(populacao)
                pai2 = random.choice(populacao)
            
            filho1, filho2 = crossover(pai1, pai2)
            nova_populacao.extend([mutacao(filho1, taxa_mutacao), mutacao(filho2, taxa_mutacao)])

        populacao = nova_populacao[:populacao_tamanho]

    # Obter a melhor solução
    fitnesses = [calcular_fitness(cromo) for cromo in populacao]
    melhor_indice = np.argmax(fitnesses)
    melhor_solucao = populacao[melhor_indice]

    return melhor_solucao, fitnesses[melhor_indice]

In [100]:
def imprimir_resultado(solucao, fitness):
    print("\n" + "="*50)
    print("RESULTADO DA OTIMIZAÇÃO".center(50))
    print("="*50)
    print(f"Fitness Final: {fitness:,}")

        # Estatísticas adicionais
    total_match_perfeito = sum(1 for os_index, op_index in enumerate(solucao)
                              if ordens_servico[os_index][1] in operadores[op_index][1])
    
    print(f"Taxa de Sucesso: {(total_match_perfeito/len(solucao))*100:.1f}%")
    print(f"Matches Perfeitos: {total_match_perfeito}/{len(solucao)}")

    print(f"Alocação de Recursos:")


    print("-" * 130)
    for os_index, operador_index in enumerate(solucao):
        os = ordens_servico[os_index]
        op = operadores[operador_index]
        status = "✓" if (os[1] in op[1] and os[2] <= op[2] and os[3] <= op[3]) else "✗"
        
        print(f"{status} OS #{os[0]:4d} (Tipo: {os[1]:10s}, Nível: {os[2]}, Prazo: {os[3]:2d}) "
                f"→ Operador #{op[0]:2d} (Nível: {op[2]}, Disp: {op[3]},  Habils: {', '.join(op[1])})")
    print("-+" * 130)

# Execução do Algoritmo
melhor_solucao, melhor_fitness = algoritmo_genetico(20000)
imprimir_resultado(melhor_solucao, melhor_fitness)


               INICIANDO OTIMIZAÇÃO               
Iniciando algoritmo genético com 20000 gerações
Estratégia: Elitismo
[Geração     0] População:  20 | Taxa Mutação: 10.00% | Melhor Fitness: 10180 | Melhoria: +0
[Geração     1] População:  20 | Taxa Mutação: 10.00% | Melhor Fitness: 10590 | Melhoria: +410
[Geração     2] População:  20 | Taxa Mutação: 10.00% | Melhor Fitness: 11280 | Melhoria: +690
[Geração     5] População:  20 | Taxa Mutação: 10.00% | Melhor Fitness: 11560 | Melhoria: +280
[Geração     9] População:  20 | Taxa Mutação: 10.00% | Melhor Fitness: 11640 | Melhoria: +80
[Geração    10] População:  20 | Taxa Mutação: 10.00% | Melhor Fitness: 11780 | Melhoria: +140
[Geração    11] População:  20 | Taxa Mutação: 10.00% | Melhor Fitness: 11790 | Melhoria: +10
[Geração    14] População:  20 | Taxa Mutação: 10.00% | Melhor Fitness: 11960 | Melhoria: +170
[Geração    21] População:  20 | Taxa Mutação: 10.00% | Melhor Fitness: 12050 | Melhoria: +90
[Geração    22] População:  2